In [1]:
from datetime import datetime, timedelta
from fractions import Fraction

import configparser
import pandas as pd

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
last_paid = datetime.strptime(config['DEFAULT']['last_paid'], '%Y-%m-%d')
mint_analysis = datetime.strptime(config['DEFAULT']['mint_analysis'], '%Y-%m-%d')

In [3]:
chase = pd.read_csv('ChaseCC.csv')
citi = pd.read_csv('CitiCC.csv')
mint = pd.read_csv('Mint.csv')

In [4]:
chase['trans_date'] = pd.to_datetime(chase['Transaction Date'])
citi['trans_date'] = pd.to_datetime(citi['Date'])
mint['trans_date'] = pd.to_datetime(mint['Date'])

In [5]:
chase = chase[chase['trans_date'] > last_paid]
chase = chase[chase['Type'] != 'Payment']
citi = citi[citi['trans_date'] > last_paid]
citi = citi[~citi['Description'].str.contains('PAYMENT, THANK YOU')]
mint = mint[mint['trans_date'] > mint_analysis]

In [6]:
chase['amt'] = chase['Amount'] * -1

In [7]:
citi.loc[(citi['Debit'].notnull()), 'amt'] = citi['Debit']
citi.loc[(citi['Credit'].notnull()), 'amt'] = citi['Credit']

In [8]:
mint.loc[(mint['Transaction Type'] == 'debit'), 'amt'] = mint['Amount']
mint.loc[(mint['Transaction Type'] == 'credit'), 'amt'] = mint['Amount'] * -1

In [9]:
chase = chase[['trans_date', 'amt', 'Description']]
citi = citi[['trans_date', 'amt', 'Description']]
mint = mint[['trans_date', 'amt', 'Category', 'Description', 'Original Description']]

In [10]:
chase.head(5)

,trans_date,amt,Description
0,2020-12-21,47.69,UBER EATS
1,2020-12-16,25.00,TVY*FITNESSYOURWAY
2,2020-12-14,32.66,UBER EATS
3,2020-12-13,42.77,UBER EATS
4,2020-12-10,114.54,SPRINT *WIRELESS


In [11]:
citi.head(5)

,trans_date,amt,Description
0,2020-12-24,-449.99,PAYPAL *B H PHOTO 4029357733 NY Digital Accoun...
1,2020-12-24,6.36,PAYPAL *AMAGICOM AB 35314369001 SWE
2,2020-12-22,115.62,TRADER JOE'S #692 QPS BRENTWOOD MO
3,2020-12-21,10.69,PAYPAL *SPOTIFYUSAI 4029357733 NY Digital Acco...
4,2020-12-19,126.14,Amazon Prime*YG8V21593 8665572820 WA


In [12]:
mint.head(5)

,trans_date,amt,Category,Description,Original Description
0,2020-12-28,115.24,Groceries,Trader Joe's,TRADER JOE'S # 692 BRENTWOOD MO
1,2020-12-28,262.66,Credit Card Payment,CHASE CREDIT CRD,ACH DEBIT CHASE CREDIT CRD
2,2020-12-28,1550.96,Credit Card Payment,CITI CARD ONLINE,ACH DEBIT CITI CARD ONLINE
3,2020-12-27,12.48,Restaurants,Uber Eats,UBER *EATS
4,2020-12-27,32.68,Restaurants,Uber.com,UBER * EATS PENDING


In [13]:
round(citi['amt'].sum(),2)

1492.11

In [14]:
income_bk = round(float(config['BUDGET']['income_bk_raw']) * float(Fraction(config['BUDGET']['income_bk_mult'])),2)
income_bk

4855.95

In [15]:
categories = pd.read_excel('budget_categories.xlsx', sheet_name='main', engine='openpyxl')
categories.sort_values(by=['sort1', 'sort2', 'sort3'], inplace=True)
categories.reset_index(drop=True, inplace=True)
categories.head(5)

,sort1,sort2,sort3,category,sub_group,group,show_cat,show_sub,show_group,necessity
0,1,0,0,Income,Income,Income,False,False,False,Ignore
1,1,1,1,Paycheck,Income,Income,True,False,False,Ignore
2,1,1,2,Bonus,Income,Income,False,False,False,Ignore
3,1,1,3,Interest Income,Income,Income,False,False,False,Ignore
4,1,1,4,Rental Income,Income,Income,False,False,False,Ignore


In [16]:
mint = mint.groupby(by=['Category']).sum()
mint.head(5)

,amt
Category,
Alcohol & Bars,26.55
Credit Card Payment,1534.63
Electric,191.94
Entertainment,45.68
Financial,982.20


In [17]:
budget_temp = categories.merge(mint, how='left', left_on='category', right_on='Category')
budget_temp.head(5)

,sort1,sort2,sort3,category,sub_group,group,show_cat,show_sub,show_group,necessity,amt
0,1,0,0,Income,Income,Income,False,False,False,Ignore,NaN
1,1,1,1,Paycheck,Income,Income,True,False,False,Ignore,-13522.30
2,1,1,2,Bonus,Income,Income,False,False,False,Ignore,NaN
3,1,1,3,Interest Income,Income,Income,False,False,False,Ignore,-36.31
4,1,1,4,Rental Income,Income,Income,False,False,False,Ignore,NaN


In [18]:
months_mint = ((datetime.today() - mint_analysis).days)/(365.25/12)
budget_temp['amt'] = round(budget_temp['amt']/months_mint,2)

In [19]:
budget_temp['amt2'] = budget_temp.groupby(['sub_group'])['amt'].transform('sum')
budget_temp['amt3'] = budget_temp.groupby(['group'])['amt'].transform('sum')
budget_temp.sort_values(by=['sort1', 'sort2', 'sort3'], inplace=True)
budget_temp.head(5)

,sort1,sort2,sort3,category,sub_group,group,show_cat,show_sub,show_group,necessity,amt,amt2,amt3
0,1,0,0,Income,Income,Income,False,False,False,Ignore,NaN,-3527.27,-3527.27
1,1,1,1,Paycheck,Income,Income,True,False,False,Ignore,-3517.82,-3527.27,-3527.27
2,1,1,2,Bonus,Income,Income,False,False,False,Ignore,NaN,-3527.27,-3527.27
3,1,1,3,Interest Income,Income,Income,False,False,False,Ignore,-9.45,-3527.27,-3527.27
4,1,1,4,Rental Income,Income,Income,False,False,False,Ignore,NaN,-3527.27,-3527.27


In [20]:
not budget_temp.loc[1, 'show_group']

True

In [21]:
budget_analysis = pd.DataFrame(columns = ['category', 'amt'])
prev_group = ''
prev_sub = ''
prev_cat = ''
count = 0
for index, row in budget_temp.iterrows():
    next_value = budget_temp.loc[index, 'group']
    if next_value == prev_group or budget_temp.loc[index, 'show_group'] == False:
        pass
    else:
        budget_analysis.loc[count, 'category'] = next_value
        budget_analysis.loc[count, 'amt'] = budget_temp.loc[index, 'amt3']
        prev_group = next_value
        count += 1
    next_value = budget_temp.loc[index, 'sub_group']
    if next_value == prev_group or next_value == prev_sub or budget_temp.loc[index, 'show_sub'] == False:
        pass
    else:
        budget_analysis.loc[count, 'category'] = next_value
        budget_analysis.loc[count, 'amt'] = budget_temp.loc[index, 'amt2']
        prev_sub = next_value
        count += 1
    next_value = budget_temp.loc[index, 'category']
    if next_value == prev_group or next_value == prev_sub or next_value == prev_cat or budget_temp.loc[index, 'show_cat'] == False:
        pass
    else:
        budget_analysis.loc[count, 'category'] = next_value
        budget_analysis.loc[count, 'amt'] = budget_temp.loc[index, 'amt']
        prev_cat = next_value
        count += 1

In [22]:
budget_analysis.sort_values(['amt'], ascending=False)

,category,amt
1,Mortgage & Rent,1716.99
29,Misc Expenses,1488.66
8,Groceries,410.37
9,Restaurants,148.05
3,Home Utilities,92.89
6,Phone,89.54
4,Electric,49.93
7,Internet,44.31
5,Natural Gas,42.96
25,Gym,19.51


In [23]:
bills = pd.read_csv('bills.csv')
bills['recur_date'] = pd.to_datetime(bills['recur_date'])

In [24]:
from dateutil.rrule import rrule, DAILY, WEEKLY, MONTHLY

for index, row in bills.iterrows():
    recur = bills.loc[index, 'recurrence']
    recur_date = bills.loc[index, 'recur_date']
    if recur == 'bi-weekly':
        temp_schedule = rrule(WEEKLY, dtstart=recur_date, count=52, interval=2)
    elif recur == 'monthly':
        temp_schedule = rrule(MONTHLY, dtstart=recur_date, count=24, interval=1)
    elif recur == 'yearly':
        temp_schedule = rrule(MONTHLY, dtstart=recur_date, count=4, interval=12)
    else:
        sys.exit('Recurrance type not defined')
    for each in temp_schedule:
        if each.date() >= (datetime.now() - timedelta(days=7)).date():
            bills.loc[index, 'recur_date'] = each.date()
            break
        else:
            pass

bills    



,category,Title,recurrence,recur_date
0,Paycheck,Paycheck - BK,bi-weekly,2021-01-21
1,Mortgage & Rent,Rent,monthly,2021-01-28
2,Electric,Ameren,monthly,2021-02-10
3,Natural Gas,Spire,monthly,2021-01-19
4,Mobile Phone,Sprint,monthly,2021-02-09
5,Internet,Spectrum,monthly,2021-01-31
6,Motorcycle Insurance,Dairyland,yearly,2021-07-16
7,Gym,FitnessYourWay,monthly,2021-02-16
8,Spotify,Spotify,monthly,2021-01-23


In [25]:
bills.to_csv('bills.csv', index=False)

In [29]:
last_next_28 = rrule(DAILY, dtstart=(datetime.now() - timedelta(days=7)).date(), count=28, interval=1)
last_7_df = pd.DataFrame(columns=['date', 'charges', 'amt'])
next_21_df = pd.DataFrame(columns=['date', 'charges', 'amt'])
for index, each in enumerate(last_next_28):
    if index <= 6:
        last_7_df.loc[index, 'date'] = each
        last_7_df.loc[index, 'charges'] = '|'
        last_7_df.loc[index, 'amt'] = 0
    else:
        next_21_df.loc[index - 7, 'date'] = each
        next_21_df.loc[index - 7, 'charges'] = '|'
        next_21_df.loc[index - 7, 'amt'] = 0

In [30]:
last_7_df

,date,charges,amt
0,2021-01-19 00:00:00,|,0
1,2021-01-20 00:00:00,|,0
2,2021-01-21 00:00:00,|,0
3,2021-01-22 00:00:00,|,0
4,2021-01-23 00:00:00,|,0
5,2021-01-24 00:00:00,|,0
6,2021-01-25 00:00:00,|,0


In [31]:
next_21_df

,date,charges,amt
0,2021-01-26 00:00:00,NaN,NaN
7,2021-02-02 00:00:00,|,0
1,2021-01-27 00:00:00,NaN,NaN
8,2021-02-03 00:00:00,|,0
2,2021-01-28 00:00:00,NaN,NaN
9,2021-02-04 00:00:00,|,0
3,2021-01-29 00:00:00,NaN,NaN
10,2021-02-05 00:00:00,|,0
4,2021-01-30 00:00:00,NaN,NaN
11,2021-02-06 00:00:00,|,0


# To-Do's
- Remove charges that already happened
- Store budget information
- Create calendar of how to predict upcoming paychecks
- Make list/calendar of upcoming expenses
- Calculate remaining budget based off upcoming expenses
- Create budget analysis (regression and past month)
- Create graphs and visuals
- Machine Learning budget